<a href="https://colab.research.google.com/github/BiakkoNoMiko/homework_projects/blob/main/Summarization_with_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install library for downloading datasets from online sources like Kaggle
!pip install opendatasets --upgrade
# Install libraries for transformers
!pip install transformers[sentencepiece] transformers[torch] datasets evaluate rouge_score sentencepiece

!pip install -q git+https://github.com/huggingface/accelerate.git

# Couldn't apply Lora for mt5-small (Nans in Losses)
#!pip install -q git+https://github.com/huggingface/peft.git
#!pip install tensorboardX==2.6
#!pip install -q loralib einops
#!pip install appdirs==1.4.4
#!pip install addict evaluate
#!pip install --upgrade git+https://github.com/lvwerra/trl
#!pip install -q -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl 

## huggingface login and GPU info

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!nvidia-smi

Fri Sep  1 11:18:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Donwload dataset
and first preparations

In [ ]:
#System and standart
import opendatasets as od
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="my bar!")


from datasets import Dataset
#from peft import (LoraConfig, TaskType,  get_peft_model, get_peft_model_state_dict, prepare_model_for_int8_training, )
#from transformers import BitsAndBytesConfig

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Downloading dataset directly from kaggle
od.download("https://www.kaggle.com/datasets/yutkin/corpus-of-russian-news-articles-from-lenta?resource=download")

100%|██████████| 584M/584M [00:31<00:00, 19.4MB/s]


## Pandas dataframe

In [ ]:
df = pd.read_csv('/content/corpus-of-russian-news-articles-from-lenta/lenta-ru-news.csv')

<ipython-input-6-741eacf25199>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/corpus-of-russian-news-articles-from-lenta/lenta-ru-news.csv')


In [ ]:
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800975 entries, 0 to 800974
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     800975 non-null  object
 1   title   800975 non-null  object
 2   text    800970 non-null  object
 3   topic   738973 non-null  object
 4   tags    773756 non-null  object
 5   date    800975 non-null  object
dtypes: object(6)
memory usage: 36.7+ MB


,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914/09/18
5,https://lenta.ru/news/1999/08/31/stancia_mir/,"Космонавты сомневаются в надежности ""Мира""",Как стало известно агентству Ассошиэйтед Пресс...,Россия,Все,1999/08/31
6,https://lenta.ru/news/1999/08/31/vzriv/,Взрыв в центре Москвы: пострадало 30 человек,В зале игровых автоматов в третьем ярусе подзе...,Россия,Все,1999/08/31
7,https://lenta.ru/news/1999/08/31/credit_japs/,Япония кредитует Россию на полтора миллиарда д...,Япония приняла решение разморозить кредиты Рос...,Россия,Все,1999/08/31
8,https://lenta.ru/news/1999/08/31/diana/,Британцы отмечают двухлетие смерти Дианы,Британцы отмечают сегодня скорбную дату - втор...,Мир,Все,1999/08/31
9,https://lenta.ru/news/1999/08/31/mvf/,Отмытые через Bank of NY деньги не имели отнош...,В понедельник директор департамента внешних св...,Россия,Все,1999/08/31


In [ ]:
# look at percentage of Nan
null_values=pd.DataFrame(df.isna().sum(), columns=['NaN'])
null_values['percent'] = (null_values['NaN'] /len(df) * 100)
null_values.sort_values('percent',ascending=False).round(2)

,NaN,percent
topic,62002,7.74
tags,27219,3.40
text,5,0.00
url,0,0.00
title,0,0.00
date,0,0.00


text column is the most important for our purposes. Let's drop all lines with Nans in the text column

In [ ]:
df.dropna(subset = ['text'], inplace=True)
df.shape

(800970, 6)

In [ ]:
df.topic.value_counts()

Россия               160442
Мир                  136620
Экономика             79528
Спорт                 64413
Культура              53796
Бывший СССР           53402
Наука и техника       53136
Интернет и СМИ        44663
Из жизни              27605
Дом                   21734
Силовые структуры     19596
Ценности               7766
Бизнес                 7399
Путешествия            6408
69-я параллель         1268
Крым                    666
Культпросвет            340
Легпром                 114
Библиотека               65
Оружие                    3
ЧМ-2014                   2
МедНовости                1
Сочи                      1
Name: topic, dtype: int64

It's a large dataset, I'm afraid it will cause lack of resources, so let's take a part of it for our purporses.

In [ ]:
df_short = df[:int(len(df)/20)]
df_short.shape

(40048, 6)

## Transformers dataset

In [ ]:
#convert dataframe to transformers dataset format
dataset = Dataset.from_pandas(df_short[['title', 'text']])

In [ ]:
dataset

Dataset({
    features: ['title', 'text', '__index_level_0__'],
    num_rows: 40048
})

In [ ]:
#split into training and testing sets
dataset = dataset.train_test_split(test_size=0.2)

In [ ]:
dataset["train"][100]

{'title': 'На юге Филиппин взяты в\xa0заложники еще 70\xa0человек',
 'text': 'В среду исламские боевики, действующие на юге Филиппин, взяли в заложники на острове Минданао еще как минимум 70 человек. Члены группировки Исламский фронт освобождения захватили два автобуса с пассажирами и скрылись в неизвестном направлении, передает France Presse со ссылкой на военные источники. Это произошло утром 3 мая, примерно в 950 километрах к югу от филиппинской столицы. Около 200 боевиков-исламистов блокировали шоссе и захватили заложников, которых они намерены использовать как живой щит, если армия попытается начать наступление на позиции повстанцев. Исламский фронт освобождения с 1978 года борется засоздание на острове Минданао независимого исламского государства.',
 '__index_level_0__': 7257}

**There** are two fields that we'll want to use:

- `text`: the text of the bill which'll be the input to the model.
- `title`: a condensed version of `text` which'll be the model target.

## Transformers model

### Preprocess

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
#find the longest title
df["title"].str.len().max()

132

In [ ]:
df.loc[df["title"].str.len().max()]['text']

'Москва может пересмотреть формы своего участия в миротворческой операции в Косово. "Если не будет прогресса в обеспечении стабильности в регионе или дело будет идти к отделению Косово от Югославии с последующим созданием моноэтнического государства, то Совет Федерации может пересмотреть формы участия России в этой операции", - заявил в среду "Интерфаксу" начальник Главного управления международного военного сотрудничества Минобороны РФ генерал-полковник Леонид Ивашов. По мнению Ивашова, пока силы НАТО в Косово должным образом не обеспечивают безопасность народов края, и прежде всего неалбанской его части. Он также не удовлетворен ходом разоружения Армии освобождения Косово (АОК) и считает, что к намеченному сроку (19 сентября) оружие сдано не будет. "Никакого серьезного разоружения АОК не происходит, поэтому говорить о какой-то стабильности в регионе бессмысленно", - подчеркнул генерал. Ивашов также назвал преждевременными "разговоры о тесном взаимодействии" российских "голубых касок"

In [ ]:
inputs = tokenizer(df.loc[df["title"].str.len().max()]['text'])
inputs

{'input_ids': [20892, 259, 3331, 2077, 16045, 8824, 433, 1498, 1802, 6277, 1419, 315, 259, 238561, 88188, 36515, 279, 315, 259, 59149, 411, 260, 313, 6835, 401, 259, 4466, 144732, 308, 315, 16567, 9264, 56620, 102540, 315, 18945, 324, 1255, 18203, 259, 4466, 31979, 1363, 778, 11307, 13397, 259, 59149, 411, 552, 7646, 111983, 3229, 388, 14193, 85060, 8046, 38449, 89843, 25537, 686, 27085, 36142, 308, 261, 1049, 28364, 28628, 279, 259, 3331, 2077, 16045, 8824, 433, 6277, 1419, 8088, 315, 1436, 543, 36515, 279, 754, 259, 264, 374, 105637, 315, 6110, 354, 313, 160522, 132258, 354, 311, 259, 61868, 75867, 893, 259, 30821, 259, 40581, 893, 992, 25461, 24818, 74985, 16778, 173983, 433, 259, 14056, 38285, 264, 205649, 93045, 74891, 22306, 1120, 70714, 685, 260, 1325, 7604, 25223, 1120, 70714, 2940, 261, 14389, 9883, 433, 2796, 38170, 315, 259, 59149, 411, 8203, 2587, 9219, 637, 401, 16567, 51116, 128446, 833, 6239, 685, 6065, 396, 261, 259, 279, 2374, 48581, 315, 17309, 401, 52537, 5084, 259, 

In [ ]:
# in tokens the length is
len(tokenizer.convert_ids_to_tokens(inputs.input_ids))

352

In [ ]:
instruction = "суммаризируй:"


def preprocess_function(examples):
    inputs = [instruction + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["title"], max_length=256, truncation=False) # max_length=128. isn't it to short for russian language? let's take 256
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/32038 [00:00<?, ? examples/s]

Map:   0%|          | 0/8010 [00:00<?, ? examples/s]

### Evaluate

 For this task, we'll load the ROUGE metric

In [ ]:
import evaluate

rouge_score = evaluate.load("rouge")

In [ ]:
from nltk.tokenize import sent_tokenize

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    result = {key: value * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

### Train

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained('biakko/mt5_summnerizing_ru_10_epochs') # first we use mt5 model here, but now we'll continue to finetune our own model

Now create a batch of examples using DataCollatorForSeq2Seq. It's more efficient to *dynamically pad* the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
tokenized_dataset = tokenized_dataset.remove_columns(dataset["train"].column_names)

In [ ]:
features = [tokenized_dataset["train"][i] for i in range(2)]
data_collator(features)

{'input_ids': tensor([[  7023,   1977,  60149, 100783,    267,   4004,   2249, 109898,   1419,
          10900,    308, 132896,  14602,    259,  17595,    259,  23676,    893,
          50258,    308,   1436,    543,    259,  29070,    315,  14455,   6150,
            374,  23472,   2683,   4614,  63765,    990,  22666,  12941,    637,
            315,  43078,    279,    315, 213012,  68909,   1293,    259,    279,
         133441,  17755,   7214,    433,    310,  22983,    279,    259,  14056,
            261,  19707,   5613,    313, 160522, 132258,   1191,   4146,    259,
           6462,    374, 105637,    259,  82058,    264,  98386,  95460, 109898,
           3229,   8244,    308,   3387, 100396,  23622,   1126,    425,  42092,
          67333,    279, 190484, 133068,    388,    259,  23676,   1104,    259,
          82058,    637,  40948,    279,  21764,  55202,   8231,  77253,    260,
           1325,  11970,    507,   8244,    308,   3387, 100396,  23622,   5172,
            26

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="mt5_summnerizing_ru_10_epochs",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=False, #mt5 produces Nans in Losses if the parameter is on
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
trainer.push_to_hub()

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

'https://huggingface.co/Biakko/mt5_summnerizing_ru_10_epochs/tree/main/'

### Inference

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="biakko/mt5_summnerizing_ru_10_epochs")

In [ ]:
from transformers import pipeline
summarizer_wo_finetune = pipeline("summarization", model="google/mt5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
text = 'По меньшей мере три человека погибли в результате взрыва на спиртозаводе "Амурский кристалл" в Благовещенске. Как сообщает НТВ, взрыв на заводе произошел в ночь на четверг, 15 ноября, в бродильном отделении цеха по производству спирта. Как сообщили журналистам сотрудники ГО и ЧС, при взрыве обрушилась половина пятиэтажного здания и начался пожар. Потушить огонь смогли только спустя несколько часов. В момент взрыва в цехах завода находилась ночная смена - 11 человек. Трое из них - две женщины и мужчина -  погибли под обломками. Тела женщин спасателям уже удалось извлечь. Начато расследование причин случившегося. По предварительной версии, взрыв произошел из-за нарушения технологического производства. По другой версии, взрыв был вызван утечкой газа, зафиксированной 14 ноября вечером сотрудниками завода. Кроме того, по информации "Эха Москвы", еще одной возможной причиной взрыва следователи признали изношенность оборудования, которое не менялось почти 100 лет.'
print(f'W/o finetuning: {summarizer_wo_finetune(text)[0]["summary_text"]}')
print(f'With finetuning: {summarizer(text)[0]["summary_text"]}')

W/o finetuning: <extra_id_0> обрушилась половина пятиэтажного здания.
With finetuning: Взрыв на спиртозаводе в Благовещенске: три человека погибли


In [ ]:
def print_summary(idx):
    review = dataset["test"][idx]["text"]
    title = dataset["test"][idx]["title"]
    summary = summarizer(dataset["test"][idx]["text"])[0]["summary_text"]
    print(f"'>>> Review: {review}'")
    print(f"\n'>>> Title: {title}'")
    print(f"\n'>>> Summary: {summary}'")

In [ ]:
print_summary(92)

'>>> Review: Российский теннисист Марат Сафин, обыграв в матче третьего раунда испанца Томми Роберто (Tommy Roberto) - 7:6, 6:1, вышел в четвертьфинал теннисного турнира RCA, сообщает Reuters. Нынешний чемпион открытого первенства США (U.S. Open), Сафин после довольно тяжелого первого сета провел практически идеальный второй, проиграв всего три очка на своей подаче. "Второй сет был практически совершенством. Я играл гораздо лучше, чем два дня назад. Возможно, я еще лучше сыграю в следующем матче", - прокомментировал свою игру Сафин. Соперником российского теннисиста в четвертьфинале будет Томас Энквист из Швеции (Thomas Enqvist), преодолевший в третьем раунде сопротивление своего соотечественника и тезки Томаса Йоханссона (Thomas Johansson).'

'>>> Title: Марат Сафин вышел в четвертьфинал теннисного турнира в Индианаполисе'

'>>> Summary: Марат Сафин вышел в четвертьфинал теннисного турнира RCA'


Manual inference

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("biakko/my_awesome_summerizing_model")
inputs = tokenizer(text, return_tensors="pt").input_ids

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("biakko/my_awesome_summerizing_model")
outputs = model.generate(inputs, max_new_tokens=50, do_sample=False)

In [ ]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

###Apply LORA

In [ ]:
# model.gradient_checkpointing_enable() # slowdown process significally
#model = prepare_model_for_int8_training(model)

#config = LoraConfig(
#    r = 16,
#    lora_alpha = 32,
#    lora_dropout=0.05,
#    bias="none",
#    task_type=TaskType.SEQ_2_SEQ_LM,
#    inference_mode=False,
#)
#model = get_peft_model(model, config)
#model.print_trainable_parameters()

In [ ]:
# Verifying the datatypes.
#dtypes = {}
#for _, p in model.named_parameters():
#    dtype = p.dtype
#    if dtype not in dtypes:
#        dtypes[dtype] = 0
#    dtypes[dtype] += p.numel()
#total = 0
#for k, v in dtypes.items():
#    total += v
#for k, v in dtypes.items():
#    print(k, v, v / total)